In [2]:
import os
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical 
# 학습률
from tensorflow.keras.optimizers import Adam
from PIL import Image
from sklearn.model_selection import train_test_split # 검증데이터 쪼개기

In [3]:
# npz 파일 로딩
data = np.load('./data/food_images.npz')

In [4]:
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

KeyboardInterrupt: 

In [9]:
folder_names = os.listdir('./data/archive/images/')
folder_names

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito',
 'bruschetta',
 'caesar_salad',
 'cannoli',
 'caprese_salad',
 'carrot_cake',
 'ceviche',
 'cheesecake',
 'cheese_plate',
 'chicken_curry',
 'chicken_quesadilla',
 'chicken_wings',
 'chocolate_cake',
 'chocolate_mousse',
 'churros',
 'clam_chowder',
 'club_sandwich',
 'crab_cakes',
 'creme_brulee',
 'croque_madame',
 'cup_cakes',
 'deviled_eggs',
 'donuts',
 'dumplings',
 'edamame',
 'eggs_benedict',
 'escargots',
 'falafel',
 'filet_mignon',
 'fish_and_chips',
 'foie_gras',
 'french_fries',
 'french_onion_soup',
 'french_toast',
 'fried_calamari',
 'fried_rice',
 'frozen_yogurt',
 'garlic_bread',
 'gnocchi',
 'greek_salad',
 'grilled_cheese_sandwich',
 'grilled_salmon',
 'guacamole',
 'gyoza',
 'hamburger',
 'hot_and_sour_soup',
 'hot_dog',
 'huevos_rancheros',
 'hummus',
 'ice_cream',
 'lasagna',
 'lobster_bisque',
 'lobster

In [10]:
base_folder_path = './data/archive/images/'
base_folder_path + folder_names[0]

'./data/archive/images/apple_pie'

In [11]:
# 전체 사진 불러오는 함수
def load_images ( folder_path,img_size) :
    images = [] # 불러올 사진을 담을 리스트
    
    file_names = os.listdir(folder_path) # 음식 폴더별 사진 이름 가져오기
    
    for i in range(len(file_names)) :
        path = folder_path + file_names[i] # 전체 풀 경로로 사진 생성
        open_image = Image.open(path).resize(img_size) # 반복문으로 생성되는 사진 경로를 찾아서 이미지를 연다, 열린 사진을 리사이징
        temp_img = np.array(open_image)
        if temp_img.shape == (128,128,3) : # 채널이 3인 사진만 저장
            images.append(np.array(open_image)) # 오픈된 사진을 numpy 배열로 변경 후 리스트에 추가
        
    return np.array(images)

In [12]:
!pip install tqdm

In [21]:
from tqdm import tqdm

In [22]:
# 경로 만들기

total_food_images = []

for i in tqdm(range(len(folder_names))) : 
    path = base_folder_path + folder_names[i] + "/"
    # 폴더별로 이미지 사진을 알려준다
    food_images =  load_images(path,(128,128)) # path를 load_image에 넣어준다.
    total_food_images.append(food_images)

 53%|█████▎    | 54/101 [03:28<03:01,  3.85s/it]


KeyboardInterrupt: 

In [16]:
total_food_images[10].shape

NameError: name 'total_food_images' is not defined

### 2 정답 데이터 생성

In [18]:
labels = []
cnt = 0
for images in total_food_images : 
#     print(images.shape)
    labels = labels + [cnt] * images.shape[0] # 각 데이터 갯수만큼 정답데이터 생성
    cnt = cnt + 1

NameError: name 'total_food_images' is not defined

In [17]:
labels[3000:]

[]

In [41]:
labels = np.array(labels)

In [42]:
labels.shape

(100997,)

In [16]:
final_food_images = np.concatenate(total_food_images) # 각 음식 numpy 배열 통합

NameError: name 'total_food_images' is not defined

In [46]:
final_food_images.shape

(100997, 128, 128, 3)

### 훈련 데이터와 평가데이터 분리

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_food_images, labels, random_state=416)

In [50]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(75747, 128, 128, 3)
(25250, 128, 128, 3)
(75747,)
(25250,)


### 4 전이학습 활용 101가지 음식 분류 모델 실습

#### 전처리

In [63]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

MemoryError: Unable to allocate 13.9 GiB for an array with shape (75747, 128, 128, 3) and data type float32

#### 정답데이터 원핫인코딩

In [62]:
y_test

array([62, 40, 58, ...,  8, 31, 40])

In [ ]:
y_train_one = to_categorical(y_train)
y_train_one[:3] # 3줄까지만 본다
# 각 아이템의 확률을 이야기 한다

### NPZ 형태로 저장 (numpy zip)

In [ ]:
 # numpy 압축 파일을 만드는 함수
np.savez_compressed('./data/food_images.npz',
                    X_train = X_train,
                    X_test = X_test,
                    y_train = y_train,
                    y_test = y_test
                    )

#### VGG 모델 전이학습

In [53]:
from tensorflow.keras.applications import VGG16

In [54]:
vgg16 = VGG16(input_shape=(128,128,3), include_top=False, weights='imagenet')

In [55]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

#### 모델의 전반부를 학습이 불가능하게 잠그기¶

In [64]:
for layer in vgg16.layers : 
    print(layer.name)
    if layer.name == 'block5_conv3' :
        layer.trainable = True
    else : 
        layer.trainable = False

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [65]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [4]:
from tensorflow.keras.models import Sequential

In [99]:
transfer_model = Sequential()
transfer_model.add(vgg16)
transfer_model.add(Flatten())
transfer_model.add(Dense(128, activation="relu"))
transfer_model.add(Dense(101,activation="softmax"))

In [100]:
transfer_model.compile(loss="categorical_crossentropy",
                      optimizer=Adam(learning_rate=0.001),
                      metrics=['accuracy'])

In [101]:
from keras.utils import to_categorical

y_train_one = to_categorical(y_train)
y_test_one = to_categorical(y_test)

In [105]:
history = transfer_model.fit(X_train,
                       y_train_one,
                        validation_split=0.2,
                        epochs=100)

Train on 60597 samples, validate on 15150 samples
Epoch 1/100
60597/60597 [==============================] - 88s 1ms/sample - loss: 0.7390 - accuracy: 0.7833 - val_loss: 5.0296 - val_accuracy: 0.2958
Epoch 2/100
60597/60597 [==============================] - 93s 2ms/sample - loss: 0.6329 - accuracy: 0.8095 - val_loss: 5.6139 - val_accuracy: 0.2929
Epoch 3/100
60597/60597 [==============================] - 92s 2ms/sample - loss: 0.5594 - accuracy: 0.8310 - val_loss: 5.7047 - val_accuracy: 0.2950
Epoch 4/100
60597/60597 [==============================] - 92s 2ms/sample - loss: 0.5057 - accuracy: 0.8460 - val_loss: 6.4743 - val_accuracy: 0.2892
Epoch 5/100
60597/60597 [==============================] - 89s 1ms/sample - loss: 0.4839 - accuracy: 0.8535 - val_loss: 6.7212 - val_accuracy: 0.2867
Epoch 6/100
60597/60597 [==============================] - 92s 2ms/sample - loss: 0.4426 - accuracy: 0.8656 - val_loss: 7.2879 - val_accuracy: 0.2886
Epoch 7/100
60597/60597 [=========================

Epoch 55/100
60597/60597 [==============================] - 92s 2ms/sample - loss: 0.2833 - accuracy: 0.9476 - val_loss: 23.1799 - val_accuracy: 0.2663
Epoch 56/100
60597/60597 [==============================] - 90s 1ms/sample - loss: 0.2815 - accuracy: 0.9475 - val_loss: 23.5404 - val_accuracy: 0.2690
Epoch 57/100
60597/60597 [==============================] - 89s 1ms/sample - loss: 0.2794 - accuracy: 0.9480 - val_loss: 23.5265 - val_accuracy: 0.2657
Epoch 58/100
60597/60597 [==============================] - 90s 1ms/sample - loss: 0.3065 - accuracy: 0.9460 - val_loss: 24.0063 - val_accuracy: 0.2663
Epoch 59/100
60597/60597 [==============================] - 89s 1ms/sample - loss: 0.2825 - accuracy: 0.9488 - val_loss: 24.1736 - val_accuracy: 0.2644
Epoch 60/100
60597/60597 [==============================] - 89s 1ms/sample - loss: 0.2904 - accuracy: 0.9496 - val_loss: 25.0313 - val_accuracy: 0.2723
Epoch 61/100
60597/60597 [==============================] - 90s 1ms/sample - loss: 0.285

In [ ]:
transfer_model.evaluate(X_test, y_test_one)

### 5 MobileNet 모델 전이학습

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [11]:
# npz 파일 로딩
data = np.load('./data/food_images.npz')

In [7]:
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [8]:
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [9]:
mbnet = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

In [10]:
mbnet.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [11]:
from keras.preprocessing.image import ImageDataGenerator

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                 random_state=419,
                                                 test_size=0.2)

In [14]:
trainDataGen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=60,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2, zoom_range=0.2)
train_iterator = trainDataGen.flow(X_train, y_train) # 변화를 줄 데이터 셋 연결

In [15]:
validDataGen = ImageDataGenerator(rescale = 1./255)
valid_iterator = validDataGen.flow(X_val, y_val) # 변화를 줄 데이터 셋 연결

In [16]:
mb_transfer_model = Sequential() # 모델을 쌓을 뼈대
mbnet.trainable = True # 모든 층이 학습가능하게 설정
mb_transfer_model.add(mbnet) #  MobileNet 추가
mb_transfer_model.add(GlobalAveragePooling2D()) # Flatten->GlobalAveragePooling
mb_transfer_model.add(Dense(128, activation='relu'))
mb_transfer_model.add(Dense(1, activation='softmax')) # 출력층 추가

In [18]:
from keras.callbacks import ModelCheckpoint
import os

In [19]:
mb_transfer_model.compile(loss="sparse_categorical_crossentropy",
                         optimizer=Adam(learning_rate=0.001),
                         metrics = ['accuracy'])


In [20]:
MODEL_DIR = './food_model'

# 폴더가 없다면 폴더 생성
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    # 저장할 파일명의 정규식 설정 {epoch의 수 - 손시값을 파일명으로 설정}

modelpath = MODEL_DIR + "/{epoch:02d}-{val_loss:.4f}.hdf5"

checkpointer = ModelCheckpoint(filepath = modelpath,
                               monitor = 'val_loss',
                               save_best_only = True,
                               verbose = 1)

In [27]:
mb_transfer_model.fit(train_iterator,
                     validation_data = valid_iterator,
                     epochs=30,
                     callbacks=[checkpointer])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1776 steps, validate for 592 steps
Epoch 1/30
   1/1776 [..............................] - ETA: 2:35:04

KeyError: 'val_loss'

In [22]:
# 학습률을 epoch가 지남에 따라 조정해주는 callback 함수
from keras.callbacks import ReduceLROnPlateau

In [30]:
reduce_lr = ReduceLROnPlateau(monitor="val_loss", # 학습률을 줄일때 참고할 값
                              patience=10, # 성능 개선여부를 판단하는 횟수
                              factor=0.1) # 10분의 1 줄어들음 0.001->0.0001 (학습률 수정 비율)
# 10번동안 validataion_loss가 개선이 안되면 iterator를 줄어드는것

In [31]:
mb_transfer_model.fit(X_train, y_train,
                     validation_data = (X_val, y_val),
                     epochs=13, # 30
                     callbacks=[checkpointer])

Train on 60597 samples, validate on 15150 samples
Epoch 1/13
60576/60597 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0101- ETA: 1s - loss
Epoch 00001: val_loss did not improve from inf
60597/60597 [==============================] - 106s 2ms/sample - loss: nan - accuracy: 0.0101 - val_loss: nan - val_accuracy: 0.0096
Epoch 2/13
60544/60597 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0101- ETA: 9s - loss: na - ETA: 8s - l - ETA: 6s - loss: nan - ac - ETA: 5s - loss: 
Epoch 00002: val_loss did not improve from inf
60597/60597 [==============================] - 102s 2ms/sample - loss: nan - accuracy: 0.0101 - val_loss: nan - val_accuracy: 0.0096
Epoch 3/13
60576/60597 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.0101
Epoch 00003: val_loss did not improve from inf
60597/60597 [==============================] - 104s 2ms/sample - loss: nan - accuracy: 0.0101 - val_loss: nan - val_accuracy: 0.0096
Epoch 4/13
60544/60597 

In [34]:
mb_transfer_model.evaluate(X_test, y_test)

25250/25250 [==============================] - 14s 561us/sample - loss: nan - accuracy: 0.0095


[nan, 0.009544554]